## Notebook to bulk download Planet imagery through the API
Rainey Aberle, Fall 2021

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and Planet Labs GitHub Repository: [planetlabs/notebooks/jupyter-notebooks/orders/](https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks/orders)


__To-Do:__
- View image footprints before downloading.  
- Implement percent AOI coverage filter for image search

### Import necessary packages

In [1]:
import os
import json
import requests
import time
import geopandas as gpd
from pathlib import Path
import rasterio as rio
import numpy as np
from rasterio.plot import show
from requests.auth import HTTPBasicAuth

### Define filters for image search
#### _Modify these sections_

In [2]:
# ----------Area of Interest (AOI)----------
#### OPTION 1: Use geojson.io to create a polygon
#    - Type geojson.io into a new browser page
#    - Draw a RECTANGULAR polygon over your AOI  
#    - Copy only the geometry below
# AOI_box = {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               -148.96774291992188,
#               60.366355109034046
#             ],
#             [
#               -148.84963989257812,
#               60.366355109034046
#             ],
#             [
#               -148.84963989257812,
#               60.440285056170616
#             ],
#             [
#               -148.96774291992188,
#               60.440285056170616
#             ],
#             [
#               -148.96774291992188,
#               60.366355109034046
#             ]
#           ]
#         ]
#       }
# AOI_box

In [3]:
#### OPTION 2: Import an existing shapefile

# Name of your file
# If your shapefile is not currently in this directory, you need to include the full file path in 'file_name' below
file_name = 'inputs/wolverineGlacier.shp'

# Read in the shapefile
AOI = gpd.read_file(file_name)

# File extension index (we don't want the .shp extension in the next line)
i = file_name.index('.shp')

# Convert to geojson
AOI.to_file(file_name[0:i]+'.geojson', driver='GeoJSON')

# Adjust AOI polygon to a rectangular shape 
# Planet only excepts a bounding BOX as a spatial filter, 
# so we need to convert our AOI to a box (if it is not already). 
AOI_box = {"type": "Polygon",
           "coordinates": [[
               [AOI.bounds.minx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.miny[0]]
           ]]
          }
AOI_box

{'type': 'Polygon',
 'coordinates': [[[-148.97493073077095, 60.35056373052069],
   [-148.83419955670448, 60.35056373052069],
   [-148.83419955670448, 60.45403189448731],
   [-148.97493073077095, 60.45403189448731],
   [-148.97493073077095, 60.35056373052069]]]}

In [4]:
# ----------AOI clipping----------
# Would you like to clip images to the AOI (True/False)?
# This greatly speeds up the ordering and downloading process.
clip_AOI = True

# ----------Date Range----------
# Format: 'YYYY-MM-DD'
start_date = "2021-01-01"
end_date = "2022-01-01"

# ----------Cloud Filter----------
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.05

# ----------Item Type----------
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
item_type = "PSScene4Band"

# ----------Asset Type----------
# Each Item Type has a number of asset types to choose from.
# Use the Item Type link above click on your Item Type to view the available Asset Types
asset_type = "analytic_sr"

# ----------Planet API Key----------
# Find your API key on your Planet Account: account.planet.com
# My Settings > API Key
# 21d92c1c372146c089f0182295d3b028
print('Enter Planet API Key: ')
API_key = input()

# ----------Output folder----------
# AKA, where you want your images to be downloaded in your directory
out_folder = '/Users/raineyaberle/Desktop/Research/PhD/Wolverine/imagery/Planet/'

Enter Planet API Key: 
21d92c1c372146c089f0182295d3b028


---------------------
### Authentication via basic HTTP

In [5]:
# set API key as environment variable
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

### Compile filters and use Quick Search to grab image IDs

In [6]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": AOI_box
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": start_date + "T00:00:00.000Z",
    "lte": end_date + "T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": max_cloud_cover
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

# define the clip tool
clip = {
    "clip": {
        "aoi": AOI_box
    }
}

# API request object
QS_request = {
  #"interval":"day",
  "item_types": [item_type], 
  "asset_types": [asset_type],
  "filter":combined_filter
}

# fire off the POST request
QS_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=QS_request)

# Print resulting image IDs
im_ids = [feature['id'] for feature in QS_result.json()['features']]
print(im_ids)

['20211119_214329_72_1058', '20211119_214328_22_1058', '20211119_214326_72_1058', '20211117_214553_20_105e', '20211117_214551_70_105e', '20211116_214233_71_1058', '20211116_214232_21_1058', '20211116_214230_71_1058', '20211115_211414_02_2406', '20211115_211411_73_2406', '20211109_210756_16_2426', '20211109_210753_86_2426', '20211109_204525_1013', '20211109_204524_1013', '20211109_201722_54_2453', '20211109_201720_06_2453', '20211109_211033_65_2403', '20211109_211031_35_2403', '20211108_202503_66_241d', '20211108_202501_19_241d', '20211108_210609_38_227a', '20211108_201956_30_2451', '20211108_204432_1039', '20211108_204431_1039', '20211108_204430_1039', '20211107_204511_101f', '20211107_204510_101f', '20211107_204509_101f', '20211106_210541_16_227c', '20211106_210538_68_227c', '20211103_204745_1009', '20211016_202323_22_245a', '20211015_214050_21_1061', '20211010_210821_73_2416', '20211010_210819_25_2416', '20211010_202330_32_2235', '20211010_202328_11_2235', '20211009_202021_01_242d', 

### Place Order

In [7]:
# set content type to json
headers = {'content-type': 'application/json'}

# create a request object
# (clip images if clip==True)
if clip_AOI:
    request = {  
       "name":"simple order",
       "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle": asset_type
          }
       ],
        "tools": [clip]
    }
else:
    request = {  
       "name":"simple order",
       "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle": asset_type
          }
       ],
    }

# define function to place order
def place_order(search_request, auth):
    response = requests.post(orders_url, data=json.dumps(search_request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

# place order
order_url = place_order(request, auth)

<Response [202]>
2ebbd8dd-21a5-45db-9b18-386e32e5adcf


### Poll for Order Success
- This section outputs the status of the order every ~10 sec. This will take a few minutes... 
- Wait until it outputs `success` to proceed to the next section. It will stop after 30 loops, so try proceeding to the next section if it finishes running and does not output `success`.
- If you are ordering a LOT of images, consider narrowing your date range to download less images at a time. 

In [12]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, auth)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


### View Results

In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

[r['name'] for r in results]

### Download each asset individually

In [ ]:
# make directory for images using image dates
out_im_path = outpath+start_date+'_'+end_date
if os.path.isdir(stitch_path)==0:
    os.mkdir(stitch_path)
    print(stitch_path+' directory made')

# define function to download results
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = Path(os.path.join(out_folder,name)) #pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
# download images!
download_results(results)

### Visualize downloaded images

In [ ]:
# define helpful functions for visualizing downloaded imagery
def show_rgb(img_file):
    with rio.open(img_file) as src:
        b,g,r,n = src.read()

    rgb = np.stack((r,g,b), axis=0)
    show(rgb/rgb.max())
    
def show_gray(img_file):
    with rio.open(img_file) as src:
        g = src.read(1)
    show(g/g.max())
    
# Replace this path with your image file path
img_file = out_folder+'2021-04-20_2021-08-25/PSScene4Band/20211007_202318_88_245c_3B_AnalyticMS_clip.tif'
show_rgb(img_file)

## You did it!

<div>
<img src="sandy-cheeks.jpeg" width="400"/>
</div>